<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise, we are using high-level abstractions from torch.nn like nn.Linear.
Note: during the next lab session we will go one level deeper and implement more things
with bare hands.

Tasks:

 1. Read the code.

 2. Check that the given implementation reaches 95% test accuracy for architecture input-128-128-10 after few epochs.

 3. Add the option to use SGD with momentum instead of ADAM.

 4. Experiment with different learning rates. Use the provided TrainingVisualizer
 to plot the learning curves and gradient-to-weight ratios. Compare visualizations
 for different learning rates for both ADAM and SGD with momentum.

 5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
 Note that this requires creating a list of layers as an attribute of the Net class,
 and one can't use a standard Python list containing nn.Modules (why?).
 Check torch.nn.ModuleList.

If you run this notebook locally then you may need to install some packages.
It may be achieved by adding the following code cell to the notebook and running it:
```
!pip install torch torchvision plotly ipywidgets
```
This notebook can also utilize Colab GPU. However, remember to kill your GPU session after classes as otherwise, you may use all your free GPU time for this week.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import plotly.graph_objects as go
import sys

if "google.colab" in sys.modules:
    from google.colab import output

    output.enable_custom_widget_manager()

In [2]:
# @title Visualize gradients


class TrainingVisualizer:
    def __init__(self, log_interval: int = 10):
        self.log_interval = log_interval
        self.train_loss_fig = self.init_line_plot(
            title="Training loss", xaxis_title="Step"
        )
        self.grad_to_weight_fig = self.init_line_plot(
            title="Gradient standard deviation to weight standard deviation ratio at 1st layer",
            xaxis_title="Step",
            yaxis_title="Gradient to weight ratio (log scale)",
            yaxis_type="log",
        )
        self.test_acc_fig = self.init_line_plot(
            title="Test accuracy", x=[], xaxis_title="Epoch", mode="lines+markers"
        )

        # Parameters related to current tracked model and its training
        self.first_linear_layer = None
        self.lr = None
        self.trace_idx = -1

    def init_line_plot(
        self,
        title: str,
        x=None,
        xaxis_title: str = None,
        yaxis_title: str = None,
        yaxis_type: str = "linear",
        mode: str = "lines",
    ):
        fig = go.Figure()
        fig.update_layout(
            title=title,
            title_x=0.5,
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            height=400,
            width=1500,
            margin=dict(b=10, t=60),
        )
        fig.update_yaxes(type=yaxis_type)
        # We cannot add new traces dynamically because Colab has a problem with widgets
        # from plotly (traces added dynamically are rendered twice).
        # As an ugly workaround we create a lot of empty traces and update them later
        # with actual data. Empty traces are not plotted.
        for _ in range(25):
            fig.add_trace(go.Scatter(x=x, y=[], showlegend=True, mode=mode))

        fig_widget = go.FigureWidget(fig)
        display(fig_widget)
        return fig_widget

    def track_model(
        self, model: torch.nn.Module, optimizer: torch.optim.Optimizer, lr: float
    ):
        """
        Start tracking training metrics for a new model.
        """

        for field in model.__dict__["_modules"].values():
            if isinstance(field, nn.Linear):
                self.first_linear_layer = field
                break
            elif isinstance(field, nn.ModuleList):
                self.first_linear_layer = field[0]
                break

        self.lr = lr
        self.trace_idx += 1

        optim_name = type(optimizer).__name__
        self.train_loss_fig.data[self.trace_idx].name = f"{optim_name}, {lr}"
        self.grad_to_weight_fig.data[self.trace_idx].name = f"{optim_name}, {lr}"
        self.test_acc_fig.data[self.trace_idx].name = f"{optim_name}, {lr}"

    def plot_gradients_and_loss(self, batch_idx: int, loss: float):
        if batch_idx % self.log_interval == 0:
            self.train_loss_fig.data[self.trace_idx].y += (loss,)

            layer = self.first_linear_layer
            grad_to_weight_ratio = (
                self.lr * layer.weight.grad.std() / layer.weight.std()
            ).item()

            self.grad_to_weight_fig.data[self.trace_idx].y += (grad_to_weight_ratio,)

    def plot_accuracy(self, epoch: int, accuracy: float):
        self.test_acc_fig.data[self.trace_idx].x += (epoch,)
        self.test_acc_fig.data[self.trace_idx].y += (accuracy,)

In [10]:
class Net(nn.Module):
    def __init__(self, layers=[784, 128, 128, 10]):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.layers = nn.ModuleList([nn.Linear(a, b) for a, b in zip(layers[:-1], layers[1:])])


    def forward(self, x: torch.Tensor):
        x = torch.flatten(x, 1)
        for l in self.layers[:-1]:
            x = l(x)
            x = F.relu(x)
        x = self.layers[-1](x)

        output = F.log_softmax(x, dim=1)
        return output


def train(
    model: torch.nn.Module,
    device: torch.device,
    train_loader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    epoch: int,
    log_interval: int,
    visualizer: TrainingVisualizer,
    verbose: bool = False,
):
    model.train()
    assert model.training
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # Let p be a parameter of our .

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        visualizer.plot_gradients_and_loss(batch_idx, loss.item())
        optimizer.step()
        if batch_idx % log_interval == 0:
            if verbose:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )


def test(
    model: torch.nn.Module,
    device: torch.device,
    test_loader: torch.utils.data.DataLoader,
    epoch: int,
    visualizer: TrainingVisualizer,
    verbose: bool = False,
):
    model.eval()
    assert not model.training
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            test_loss += F.nll_loss(
                output, target, reduction="sum"
            ).item()  # this sum up batch loss, however usually we prefer a mean reduction
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    if verbose:
        print(
            "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
                test_loss,
                correct,
                len(test_loader.dataset),
                100.0 * correct / len(test_loader.dataset),
            )
        )
    visualizer.plot_accuracy(epoch, 100.0 * correct / len(test_loader.dataset))

In [11]:
# training uses more memory than test due to gradient computation
# therefore we can set test_batch_size to be larger
batch_size = 256
test_batch_size = 1000
epochs = 5
lr = 1e-2
seed = 1
log_interval = 10
# to check whether a cuda device is available
use_cuda = torch.cuda.is_available()

In [12]:
# for reproductibility
torch.manual_seed(seed)
# if we have cuda capable device (ex, nvidia GPU) then we use it
# otherwise we use cpu, there are also other devices like mps,
# but their support for operations may be limited
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": test_batch_size}
if use_cuda:
    cuda_kwargs = {"num_workers": 1, "pin_memory": True, "shuffle": True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [13]:
# data loader preparation
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
)
dataset1 = datasets.MNIST("../data", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("../data", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [14]:
visualizer = TrainingVisualizer(log_interval=log_interval)

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e2e76adf-59d7-41ba-9aa2-4c58742f58df',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '5c87e2a1-49ec-459c-ae4b-7f2987a1686d',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '18411089-963e-4b18-82c8-56df902f4f89',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'a29b392a-73a5-4016-8975-2c2ec4573c47',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f05feff4-f2bd-4e9f-b3d0-624c086117e4',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '8bcd737d-6bb7-4ce7-9c85-15da643f4cdf',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ef0122b0-df41-48a6-b59c-df1501750140',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'e9512d69-b8a6-4872-b1fa-ff5848b62b01',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '66b07985-4613-4a80-9d71-d98407744099',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
              'type': 'scatter',
              'uid': '31dbdd3f-51a5-41eb-aaba-1a6e8856e2b5',
              'y': []},
             {'mode': 'lines',
              'showlegend': True,
     

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f46668d0-da16-4a3f-b151-eb49ee878248',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f404e25d-22fd-48f6-89b5-791181556870',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '0e3e0e31-ea75-4a31-8c3b-cf6ac1c31b4a',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '206e4ed6-93e9-4f23-9f55-9c881ba73d8d',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7

In [ ]:
# Note that there is a difference between taking tensor to device and model to device.
# Consider writing model.to(torch.device("cpu")) and some_tesnor.to(torch.device("cpu"))
# What seems to be happening in-place?
params = [
    ('adam', 1e-2),
    ('adam', 1e-3),
    ('adam', 1e-4),
    ('momentum', 1e-2),
    ('momentum', 1e-3),
    ('momentum', 1e-4),
]

for p in params:
  model = Net([784, 128, 128, 10]).to(device)
  # Look at the output of list(model.parameters()) as you add new parameters to the model
  # What happens when the parameters are inside the standard list instead of torch.nn.ModuleList?
  if p[0] == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=lr)
  if p[0] == 'momentum':
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

  visualizer.track_model(model, optimizer, p[1])

  print_details = False  # change if you want additional output

  for epoch in range(1, epochs + 1):
      train(
          model,
          device,
          train_loader,
          optimizer,
          epoch,
          log_interval,
          visualizer,
          verbose=print_details,
      )
      test(model, device, test_loader, epoch, visualizer, verbose=print_details)

# investigate the results, what is the shape of test accuracy curve? Try to explain it.